In [1]:
import matplotlib.pyplot as plt
from matplotlib.colors import CenteredNorm
from scipy.stats import spearmanr
import numpy as np
import pandas as pd
import deconomix

In [2]:
test, train = deconomix.utils.load_example()

# Prepare Test Data
# For each cell type (column), randomly select 50 genes and apply a random regulation factor (between 0.1 and 10)

test_regulated = test.copy()
np.random.seed(42)  # For reproducibility

for celltype in test_regulated.columns.unique():
    genes = test_regulated.index
    selected_genes = np.random.choice(genes, size=200, replace=False)
    regulation_factors = np.random.uniform(0.1, 10, size=200)
    test_regulated.loc[selected_genes, celltype] = (
        test_regulated.loc[selected_genes, celltype].values * regulation_factors[:, np.newaxis]
    )




Raw data already downloaded
Importing Data to Python ...
Using cached, preprocessed data
Done


In [3]:
print(test.size - (test==test_regulated).sum().sum())

47799


In [4]:
X_ref, Y_mat, C_mat = deconomix.utils.simulate_data(train, n_mixtures = 1000, n_cells_in_mix = 100)
_, Y_test, C_test = deconomix.utils.simulate_data(test_regulated, n_mixtures = 1000, n_cells_in_mix = 100)
model0 = deconomix.methods.DTD(X_ref, Y_mat, C_mat)
model0.run(iterations = 300)

  0%|          | 0/300 [00:00<?, ?it/s]

In [5]:
model_HPS = deconomix.experimental.HPS2(X_ref, Y_test, model0.gamma)

Preparing Job List
Preparing Baseline Model


  0%|          | 0/1000 [00:00<?, ?it/s]

In [6]:
model_HPS.run(n_workers=4)

Running jobs:   0%|          | 0/55 [02:26<?, ?it/s]Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Process ForkPoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
    self

KeyboardInterrupt: 